In [1]:
# Importing the libraries
from bs4 import BeautifulSoup as BS
import pandas as pd
import numpy as np
import requests 
import re


#getting all webpages of each county in the Bay area

In [2]:

 l = [# 'https://www.redfin.com/city/17151/CA/San-Francisco' 
    #  'https://www.redfin.com/county/303/CA/Alameda-County',
      'https://www.redfin.com/county/323/CA/Marin-County',
      'https://www.redfin.com/county/330/CA/Napa-County',
      'https://www.redfin.com/county/345/CA/Santa-Clara-County'

    ]

# Retreiving the multiple page links for the above counties from the website
county_urls =[]
for cl in l:
    c_textdata = requests.get(cl,headers={'User-Agent': 'Mozilla/5.0'}).text
    c_soup_parsed = BS(c_textdata,'lxml') 
    page_no=int(c_soup_parsed.find('span',attrs={'class':'pageText'}).text.split('of ')[1])
    county_urls.append(cl)
    if page_no>1:
        for n in range(2,page_no+1):
            str_no=str(n)
            new_cl = cl+'/page-'+str_no
            county_urls.append(new_cl)

county_urls

['https://www.redfin.com/county/323/CA/Marin-County',
 'https://www.redfin.com/county/323/CA/Marin-County/page-2',
 'https://www.redfin.com/county/323/CA/Marin-County/page-3',
 'https://www.redfin.com/county/323/CA/Marin-County/page-4',
 'https://www.redfin.com/county/323/CA/Marin-County/page-5',
 'https://www.redfin.com/county/323/CA/Marin-County/page-6',
 'https://www.redfin.com/county/323/CA/Marin-County/page-7',
 'https://www.redfin.com/county/330/CA/Napa-County',
 'https://www.redfin.com/county/330/CA/Napa-County/page-2',
 'https://www.redfin.com/county/330/CA/Napa-County/page-3',
 'https://www.redfin.com/county/330/CA/Napa-County/page-4',
 'https://www.redfin.com/county/330/CA/Napa-County/page-5',
 'https://www.redfin.com/county/330/CA/Napa-County/page-6',
 'https://www.redfin.com/county/330/CA/Napa-County/page-7',
 'https://www.redfin.com/county/330/CA/Napa-County/page-8',
 'https://www.redfin.com/county/330/CA/Napa-County/page-9',
 'https://www.redfin.com/county/345/CA/Santa-Cl

In [3]:
# Dictionary creation with key - county , value - List of properties
# Extracting all links in a webpage using a href tags and using regex to identify the links for properties

Prop_set={}

for url in county_urls:
    regex = r"^/CA" 
    county_url = url
    county_name_list = re.findall('CA/([^/]+)',county_url)
    text = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'}).text
    soup = BS(text,'lxml')
    all_links = [a['href'] for a in soup('a') if a.has_attr('href')] 
    one_county_prop_url = [link for link in all_links if re.match(regex,link)]
    for a in range(0, len(one_county_prop_url)):
        one_county_prop_url[a]='https://www.redfin.com'+one_county_prop_url[a]
    final_url =[]
    for l in one_county_prop_url:
        if l not in final_url:
            final_url.append(l)
    for x in county_name_list:
        if x in Prop_set:
            Prop_set[x]=Prop_set[x]+final_url
        else:
            Prop_set[x]=final_url
        
Prop_set

{'Marin-County': ['https://www.redfin.com/CA/Sausalito/6-Issaquah-Dock-94965/home/178989787',
  'https://www.redfin.com/CA/San-Rafael/712-Tamarack-Dr-94903/home/1235127',
  'https://www.redfin.com/CA/San-Rafael/41-Bonnie-Banks-Way-94901/home/1032981',
  'https://www.redfin.com/CA/San-Rafael/777-Grand-Ave-94901/unit-204/home/176155816',
  'https://www.redfin.com/CA/San-Rafael/100-Thorndale-Dr-94903/unit-137/home/703795',
  'https://www.redfin.com/CA/Novato/510-Midway-Blvd-94947/home/1342435',
  'https://www.redfin.com/CA/Novato/848-Diablo-Ave-94947/unit-3/home/994157',
  'https://www.redfin.com/CA/Novato/19-Turner-Dr-94949/home/1050684',
  'https://www.redfin.com/CA/San-Rafael/400-Deer-Valley-Rd-94903/unit-2J/home/1226707',
  'https://www.redfin.com/CA/San-Anselmo/10-Broadmoor-Ave-94960/home/883877',
  'https://www.redfin.com/CA/Mill-Valley/73-California-Ave-94941/home/934567',
  'https://www.redfin.com/CA/San-Rafael/763-Deer-Valley-Rd-94903/home/844507',
  'https://www.redfin.com/CA/Co

In [4]:
# Extracting property information from above links using tags like div, span, etc

Street= []
City = []
State = []
Zipcode = []
Price = []
Beds = []
Bath = []
Sqft = []
Property_Type = []
Year_Built = []
Status = []
Walk_Score = []
Bike_Score = []
Acre=[]
count=0

for county in Prop_set:
    for prop in Prop_set[county]:
        print(count)
        count=count+1
        al_url = prop
        al_text = requests.get(al_url,headers={'User-Agent': 'Mozilla/5.0'}).text
        al_soup = BS(al_text,'lxml')

# Using try & except blocks to identify errors 

        try:
            street = al_soup.find('div',attrs={'class':'street-address'}).text.split(',')[0]
        except:
            street=np.nan
            print(al_url)
            #print('street')
        Street.append(street)
        
        try:
            city = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[0]
        except:
            city=np.nan
            print(al_url)
            #print('city')
        City.append(city)
        
        try:
            state = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[1].split(' ')[0]
        except:
            state=np.nan
            print(al_url)
            #print('state')
        State.append(state)
        
        try:
            zipcode = al_soup.find('div',attrs={'class':'dp-subtext'}).text.split(', ')[-1].split(' ')[-1]
            
        except:
            zipcode=np.nan
            print(al_url)
            #print('zipcode')
        Zipcode.append(zipcode)    
        
        try:
            price = al_soup.find('div',attrs={'class':'statsValue'}).text.split('$')[1]
        except:
            price=np.nan
            print(al_url)
            #print('price')
        Price.append(price)
        
        try:
            bed_str = al_soup.find('div',attrs={'data-rf-test-id':'abp-beds'}).text.split('B')[0]
            beds = int(bed_str)
        except:
            beds=np.nan
            print(al_url)
            #print('beds')
        Beds.append(beds)
        
        try:
            bath_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-baths'}).text.split('B')[0]
            bath = float(bath_str)
        except:
            bath=np.nan
            print(al_url)
            #print('bath')
        Bath.append(bath)
        
        try:
            sqft_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('S')[0].replace(',','')
            sqft = float(sqft_str)
        except:
            sqft=np.nan
            print(al_url)
            #print('sqft')
        Sqft.append(sqft)
        
        try:
            acre_str =al_soup.find('div',attrs ={'data-rf-test-id':'abp-sqFt'}).text.split('A')[0]
            acre = float(acre_str)
        except:
            acre=np.nan
            print(al_url)
            #print('acre')
        Acre.append(acre)
        
        
        try:
            div_prop = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            property_type =''
            for tag in div_prop:
                if(tag.span.text=='Property Type'):
                    property_type = property_type+tag.find('span',attrs={'class':'content text-right'}).text
        except:
            property_type = np.nan
            print(al_url)
            #print('prop_type')
        Property_Type.append(property_type)
        
        try:
            div_year_built = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            year_built =''
            for tag in div_year_built:
                if(tag.span.text=='Year Built'):
                    year_built = int(year_built+tag.find('span',attrs={'class':'content text-right'}).text)
        except:
            year_built=np.nan
            print(al_url)
            #print('year')
        Year_Built.append(year_built)
        
        try:
            div_status = al_soup.findAll('div',attrs={'class':'keyDetail font-weight-roman font-size-base'})
            status =''
            for tag in div_status:
                if(tag.span.text=='Status'):
                    status = tag.find('span',attrs={'class':'DefinitionFlyoutLink inline-block underline clickable'})
                    if status == None:
                        status = tag.find('span',attrs={'class':'content text-right'})
        except:
            status=np.nan
            print(al_url)
            #print('status')
        Status.append(status.text)

        try:
            walk_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage walkscore'}).text.split(' /')[0])
        except:
            walk_score=np.nan
            print(al_url)
            #print('walk')
        Walk_Score.append(walk_score)
        
        try:
            bike_score = int(al_soup.find('div',attrs={'class':'transport-icon-and-percentage bikescore'}).text.split(' /')[0])
        except:
            bike_score=np.nan
            print(al_url)
            #print('bike')
        Bike_Score.append(bike_score)
       


0
https://www.redfin.com/CA/Sausalito/6-Issaquah-Dock-94965/home/178989787
1
https://www.redfin.com/CA/San-Rafael/712-Tamarack-Dr-94903/home/1235127
2
https://www.redfin.com/CA/San-Rafael/41-Bonnie-Banks-Way-94901/home/1032981
3
https://www.redfin.com/CA/San-Rafael/777-Grand-Ave-94901/unit-204/home/176155816
4
https://www.redfin.com/CA/San-Rafael/100-Thorndale-Dr-94903/unit-137/home/703795
5
https://www.redfin.com/CA/Novato/510-Midway-Blvd-94947/home/1342435
6
https://www.redfin.com/CA/Novato/848-Diablo-Ave-94947/unit-3/home/994157
7
https://www.redfin.com/CA/Novato/19-Turner-Dr-94949/home/1050684
8
https://www.redfin.com/CA/San-Rafael/400-Deer-Valley-Rd-94903/unit-2J/home/1226707
9
https://www.redfin.com/CA/San-Anselmo/10-Broadmoor-Ave-94960/home/883877
10
https://www.redfin.com/CA/Mill-Valley/73-California-Ave-94941/home/934567
11
https://www.redfin.com/CA/San-Rafael/763-Deer-Valley-Rd-94903/home/844507
12
https://www.redfin.com/CA/Corte-Madera/30-Laurel-Dr-94925/home/576663
13
https

https://www.redfin.com/CA/Novato/604-Canyon-Rd-94947/home/964382
100
https://www.redfin.com/CA/Larkspur/647-Larkspur-Plaza-Dr-94939/home/747667
101
https://www.redfin.com/CA/San-Rafael/100-Thorndale-Dr-94903/unit-436/home/1026687
102
https://www.redfin.com/CA/San-Rafael/1442-Nye-St-94901/home/1023311
103
https://www.redfin.com/CA/San-Rafael/312-Catalina-Blvd-94901/home/1310984
104
https://www.redfin.com/CA/Larkspur/75-Murray-Ave-94939/home/567261
105
https://www.redfin.com/CA/Novato/82-Gazania-Ct-94945/home/944948
106
https://www.redfin.com/CA/Mill-Valley/737-Miller-Ave-94941/home/1414733
107
https://www.redfin.com/CA/Mill-Valley/50-Altamont-Ave-94941/home/1370486
108
https://www.redfin.com/CA/San-Rafael/11-Riviera-Pl-94901/home/823662
109
https://www.redfin.com/CA/Mill-Valley/526-Panoramic-Hwy-94941/home/180295189
https://www.redfin.com/CA/Mill-Valley/526-Panoramic-Hwy-94941/home/180295189
https://www.redfin.com/CA/Mill-Valley/526-Panoramic-Hwy-94941/home/180295189
110
https://www.red

https://www.redfin.com/CA/Tiburon/23-Venado-Dr-94920/home/1000380
172
https://www.redfin.com/CA/Mill-Valley/141-Great-Circle-Dr-94941/home/1159355
173
https://www.redfin.com/CA/Mill-Valley/56-Mountain-Ln-94941/home/17546762
https://www.redfin.com/CA/Mill-Valley/56-Mountain-Ln-94941/home/17546762
https://www.redfin.com/CA/Mill-Valley/56-Mountain-Ln-94941/home/17546762
174
https://www.redfin.com/CA/Tiburon/2215-Vistazo-St-E-94920/home/22326746
175
https://www.redfin.com/CA/Novato/1604-Vallejo-Ave-94945/unit-4/home/182324999
176
https://www.redfin.com/CA/San-Rafael/100-Thorndale-Dr-94903/unit-454/home/707007
177
https://www.redfin.com/CA/San-Rafael/100-Thorndale-Dr-94903/unit-349/home/1364127
178
https://www.redfin.com/CA/San-Anselmo/745-Redwood-Rd-94960/home/172534419
179
https://www.redfin.com/CA/Novato/514-Entrada-Dr-94949/unit-202/home/864451
180
https://www.redfin.com/CA/Novato/306-Rockview-Rd-94949/home/178104953
181
https://www.redfin.com/CA/Sausalito/615-Sausalito-Blvd-94965/home/

https://www.redfin.com/CA/San-Rafael/100-Thorndale-Dr-94903/unit-136/home/1136382
243
https://www.redfin.com/CA/San-Rafael/110-Sunny-Oaks-Dr-94903/home/168647133
https://www.redfin.com/CA/San-Rafael/110-Sunny-Oaks-Dr-94903/home/168647133
https://www.redfin.com/CA/San-Rafael/110-Sunny-Oaks-Dr-94903/home/168647133
244
https://www.redfin.com/CA/San-Anselmo/1-Sacramento-Ave-94960/home/22380421
https://www.redfin.com/CA/San-Anselmo/1-Sacramento-Ave-94960/home/22380421
https://www.redfin.com/CA/San-Anselmo/1-Sacramento-Ave-94960/home/22380421
245
https://www.redfin.com/CA/Nicasio/333-Willow-Rd-94946/home/1291898
https://www.redfin.com/CA/Nicasio/333-Willow-Rd-94946/home/1291898
246
https://www.redfin.com/CA/Mill-Valley/120-Redwood-Ave-94941/home/22328354
https://www.redfin.com/CA/Mill-Valley/120-Redwood-Ave-94941/home/22328354
https://www.redfin.com/CA/Mill-Valley/120-Redwood-Ave-94941/home/22328354
247
https://www.redfin.com/CA/Belvedere-Tiburon/2800-Paradise-Dr-94920/home/147283962
https:/

https://www.redfin.com/CA/Napa/230-Wintergreen-Cir-94558/home/12220179
https://www.redfin.com/CA/Napa/230-Wintergreen-Cir-94558/home/12220179
307
https://www.redfin.com/CA/Saint-Helena/216-Knoll-Pl-94574/home/12199894
308
https://www.redfin.com/CA/Angwin/835-White-Cottage-Rd-N-94508/home/184196520
309
https://www.redfin.com/CA/American-Canyon/3302-Broadway-St-94503/home/12218862
310
https://www.redfin.com/CA/Unknown/Aetna-Springs-Rd-94567/home/17546140
https://www.redfin.com/CA/Unknown/Aetna-Springs-Rd-94567/home/17546140
https://www.redfin.com/CA/Unknown/Aetna-Springs-Rd-94567/home/17546140
https://www.redfin.com/CA/Unknown/Aetna-Springs-Rd-94567/home/17546140
311
https://www.redfin.com/CA/Calistoga/25-View-Rd-94515/home/12195508
312
https://www.redfin.com/CA/Napa/131-Bonnie-Brook-Dr-94558/home/12219225
313
https://www.redfin.com/CA/American-Canyon/161-Frontier-St-94503/home/184178204
314
https://www.redfin.com/CA/American-Canyon/127-Frontier-St-94503/home/184178203
315
https://www.re

...

https://www.redfin.com/CA/Pope-Valley/2065-Stagecoach-Canyon-Rd-94567/home/12197394
https://www.redfin.com/CA/Pope-Valley/2065-Stagecoach-Canyon-Rd-94567/home/12197394
546
https://www.redfin.com/CA/Napa/405-Woodhaven-Ct-94558/home/12198395
https://www.redfin.com/CA/Napa/405-Woodhaven-Ct-94558/home/12198395
https://www.redfin.com/CA/Napa/405-Woodhaven-Ct-94558/home/12198395
https://www.redfin.com/CA/Napa/405-Woodhaven-Ct-94558/home/12198395
547
https://www.redfin.com/CA/Napa/208-Auberge-Path-94559/unit-12/home/178968844
https://www.redfin.com/CA/Napa/208-Auberge-Path-94559/unit-12/home/178968844
548
https://www.redfin.com/CA/Napa/814-Bahia-Vista-Ct-94558/home/171485392
https://www.redfin.com/CA/Napa/814-Bahia-Vista-Ct-94558/home/171485392
https://www.redfin.com/CA/Napa/814-Bahia-Vista-Ct-94558/home/171485392
https://www.redfin.com/CA/Napa/814-Bahia-Vista-Ct-94558/home/171485392
549
https://www.redfin.com/CA/Saint-Helena/157-Knoll-Pl-94574/home/12199909
550
https://www.redfin.com/CA/Sain

https://www.redfin.com/CA/Napa/Trio/Haystacks/home/180777465
587
https://www.redfin.com/CA/Napa/Trio/Pinot-Bungalow/home/180777466
588
https://www.redfin.com/CA/Napa/Trio/Napa-Vineyard/home/182581640
589
https://www.redfin.com/CA/Napa/Trio/Napa-Farmer/home/182878379
590
https://www.redfin.com/CA/Santa-Rosa/421-Trail-Ridge-Pl-95409/home/2246484
591
https://www.redfin.com/CA/Napa/1089-Ross-Cir-94558/home/12214816
592
https://www.redfin.com/CA/Kelseyville/10512-Rosa-Trl-95451/home/45694301
https://www.redfin.com/CA/Kelseyville/10512-Rosa-Trl-95451/home/45694301
593
https://www.redfin.com/CA/Clearlake/15985-41st-Ave-95422/home/45679875
594
https://www.redfin.com/CA/Rohnert-Park/7328-Circle-Dr-94928/home/2546948
595
https://www.redfin.com/CA/Santa-Rosa/3736-Woodbourne-Pl-95403/home/2614325
596
https://www.redfin.com/CA/Cobb/13377-Prather-Way-95426/home/45691731
597
https://www.redfin.com/CA/Vallejo/17-N-Daniels-Ave-94590/home/2201827
598
https://www.redfin.com/CA/Vacaville/1477-Alamo-Dr-956

https://www.redfin.com/CA/Milpitas/192-Lawton-Dr-95035/home/1626184
681
https://www.redfin.com/CA/Santa-Clara/2887-Kearney-Ave-95051/home/1304303
682
https://www.redfin.com/CA/San-Jose/5403-Colony-Park-Cir-95123/home/636991
683
https://www.redfin.com/CA/Sunnyvale/600-E-Weddell-Dr-94089/unit-249/home/584038
684
https://www.redfin.com/CA/Santa-Clara/423-Dayton-Ave-95051/home/1330732
685
https://www.redfin.com/CA/San-Jose/191-El-Bosque-Dr-95134/unit-191/home/104205404
686
https://www.redfin.com/CA/San-Jose/1233-Avenida-Benito-95131/home/12179935
687
https://www.redfin.com/CA/San-Jose/132-Chateau-La-Salle-Dr-95111/unit-132/home/184474370
688
https://www.redfin.com/CA/Campbell/2210-Central-Park-Dr-95008/home/1402603
689
https://www.redfin.com/CA/San-Jose/852-Tamarack-Ave-95128/home/1836685
690
https://www.redfin.com/CA/San-Jose/5630-Allen-Ave-95123/unit-3/home/997213
691
https://www.redfin.com/CA/San-Jose/196-S-Cragmont-Ave-95127/home/930759
692
https://www.redfin.com/CA/Sunnyvale/544-Fort-

https://www.redfin.com/CA/San-Jose/1476-Portobelo-Dr-95118/home/923736
784
https://www.redfin.com/CA/Campbell/4016-Sadie-Ct-95008/home/816699
785
https://www.redfin.com/CA/Los-Gatos/14685-Oka-Rd-95032/unit-19/home/645661
786
https://www.redfin.com/CA/Los-Gatos/16384-Peacock-Ln-95032/home/1434888
787
https://www.redfin.com/CA/Santa-Clara/4372-Laird-Cir-95054/home/1051595
788
https://www.redfin.com/CA/Campbell/182-Poplar-Ave-95008/home/1095811
https://www.redfin.com/CA/Campbell/182-Poplar-Ave-95008/home/1095811
789
https://www.redfin.com/CA/Sunnyvale/1201-Sycamore-Ter-94086/unit-122/home/1532235
790
https://www.redfin.com/CA/Sunnyvale/871-Laburnum-Dr-94086/home/1103286
791
https://www.redfin.com/CA/San-Jose/6305-Blauer-Ln-95135/home/1160308
792
https://www.redfin.com/CA/Santa-Clara/1400-Bowe-Ave-95051/unit-1504/home/735369
793
https://www.redfin.com/CA/San-Jose/1233-Lockhaven-Way-95129/home/1401911
794
https://www.redfin.com/CA/San-Jose/2584-Gimelli-Way-95133/home/172937449
795
https://w

https://www.redfin.com/CA/San-Jose/5606-Creekview-Meadow-Ln-95135/home/988331
872
https://www.redfin.com/CA/San-Jose/1044-Belvedere-Ln-95129/home/1192539
873
https://www.redfin.com/CA/Gilroy/725-Dartmouth-Pl-95020/home/1023690
874
https://www.redfin.com/CA/Santa-Clara/1167-Fairfield-Ave-95050/home/715277
875
https://www.redfin.com/CA/Santa-Clara/2233-Benton-St-95050/home/734368
876
https://www.redfin.com/CA/San-Jose/58-Saron-Ave-95116/home/1153191
877
https://www.redfin.com/CA/Morgan-Hill/744-Mendecino-Way-95037/home/909567
878
https://www.redfin.com/CA/San-Jose/3001-Ruby-Ave-95135/home/95955694
879
https://www.redfin.com/CA/San-Jose/5568-Snowdon-Pl-95138/home/12177315
880
https://www.redfin.com/CA/Cupertino/19964-Portal-Plz-95014/home/784736
881
https://www.redfin.com/CA/Santa-Clara/456-N-Winchester-Blvd-95050/home/1075473
882
https://www.redfin.com/CA/San-Jose/17231-Crothers-Rd-95127/home/948977
883
https://www.redfin.com/CA/Milpitas/1515-N-Milpitas-Blvd-95035/unit-117/home/184318806

In [5]:
County=[]
for key in Prop_set:
    for value in Prop_set[key]:
       County.append(key)
np.unique(County)

array(['Marin-County', 'Napa-County', 'Santa-Clara-County'], dtype='<U18')

In [6]:
# dataframe of all the above records

In [7]:
df=pd.DataFrame(data={'State':State,
                      'County':County,
                      'Street':Street,
                      'City':City,
                      'Zipcode':Zipcode,
                      'Price':Price,
                      'WalkScore':Walk_Score,
                      'Bike_Score':Bike_Score,
                      'Beds':Beds,
                      'Bath':Bath,
                      'Sqft':Sqft,
                      'Acre':Acre,
                      'Property_Type':Property_Type,
                      'Year_Built':Year_Built,
                      'Status':Status})

In [8]:
df     # print current dynamic data

,State,County,Street,City,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Acre,Property_Type,Year_Built,Status
0,CA,Marin-County,6 Issaquah Dock,Sausalito,94965,"1,495,000",67,83.0,2.0,2.0,1228.0,NaN,Mobile/Manufactured Home,1987,Active
1,CA,Marin-County,712 Tamarack Dr S,San Rafael,94903,"1,699,000",38,27.0,5.0,3.0,2306.0,NaN,Single Family Residential,1963,Active
2,CA,Marin-County,41 Bonnie Banks Way,San Rafael,94901,"1,995,000",29,29.0,4.0,3.0,3500.0,NaN,Single Family Residential,1970,Active
3,CA,Marin-County,777 Grand Ave Unit -204,San Rafael,94901,"560,000",86,79.0,1.0,1.0,1066.0,NaN,Condo,1988,Active
4,CA,Marin-County,100 Thorndale Dr #137,San Rafael,94903,"275,000",35,26.0,1.0,1.0,790.0,NaN,Condo,1985,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
965,CA,Santa-Clara-County,926 Northrup St,SAN JOSE,95126,"1,249,000",82,75.0,2.0,2.0,1934.0,NaN,Duplex,1957,Active
966,CA,Santa-Clara-County,4895 Pearl Ave,SAN JOSE,95136,"1,880,000",74,74.0,2.0,1.0,1322.0,NaN,Single Family Home,1917,Active
967,CA,Santa-Clara-County,15600 Peach Hill Rd,SARATOGA,95070,"3,749,000",0,NaN,3.0,2.0,2607.0,NaN,Single Family Home,1960,Active
968,CA,Santa-Clara-County,110 Cochrane Rd,MORGAN HILL,95037,"920,000",45,73.0,3.0,3.0,1773.0,NaN,Townhouse,2014,Active


In [9]:
df.isna().sum()

State              0
County             0
Street             0
City               0
Zipcode            0
Price              0
WalkScore          0
Bike_Score        94
Beds             175
Bath             175
Sqft             142
Acre             837
Property_Type      0
Year_Built         1
Status             0
dtype: int64

### Data cleaning

In [10]:
(df['Beds'].isna() & df['Bath'].isna()).sum()

175

In [11]:
 

df = df.dropna(subset=['Beds','Bath'],how='all')

In [12]:

df=df.dropna(subset=['Street','City','State','Zipcode','Price'])

In [13]:

df['WalkScore'].fillna(0,inplace=True)
df['Bike_Score'].fillna(0,inplace=True)

In [14]:

df['Beds'].fillna(0,inplace=True)
df['Bath'].fillna(0,inplace=True)

In [15]:
df.isna().sum()

State              0
County             0
Street             0
City               0
Zipcode            0
Price              0
WalkScore          0
Bike_Score         0
Beds               0
Bath               0
Sqft               4
Acre             795
Property_Type      0
Year_Built         1
Status             0
dtype: int64

In [16]:
# Dropping the rows where sqft has null values
df=df.dropna(subset=['Sqft'])

In [17]:
# removing the acre column. 
df=df.drop(columns=['Acre'])
df=df.reset_index()
df=df.drop(columns=['index'])
df

,State,County,Street,City,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,CA,Marin-County,6 Issaquah Dock,Sausalito,94965,"1,495,000",67,83.0,2.0,2.0,1228.0,Mobile/Manufactured Home,1987,Active
1,CA,Marin-County,712 Tamarack Dr S,San Rafael,94903,"1,699,000",38,27.0,5.0,3.0,2306.0,Single Family Residential,1963,Active
2,CA,Marin-County,41 Bonnie Banks Way,San Rafael,94901,"1,995,000",29,29.0,4.0,3.0,3500.0,Single Family Residential,1970,Active
3,CA,Marin-County,777 Grand Ave Unit -204,San Rafael,94901,"560,000",86,79.0,1.0,1.0,1066.0,Condo,1988,Active
4,CA,Marin-County,100 Thorndale Dr #137,San Rafael,94903,"275,000",35,26.0,1.0,1.0,790.0,Condo,1985,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,CA,Santa-Clara-County,926 Northrup St,SAN JOSE,95126,"1,249,000",82,75.0,2.0,2.0,1934.0,Duplex,1957,Active
787,CA,Santa-Clara-County,4895 Pearl Ave,SAN JOSE,95136,"1,880,000",74,74.0,2.0,1.0,1322.0,Single Family Home,1917,Active
788,CA,Santa-Clara-County,15600 Peach Hill Rd,SARATOGA,95070,"3,749,000",0,0.0,3.0,2.0,2607.0,Single Family Home,1960,Active
789,CA,Santa-Clara-County,110 Cochrane Rd,MORGAN HILL,95037,"920,000",45,73.0,3.0,3.0,1773.0,Townhouse,2014,Active


In [18]:

for i in range(0,len(df)):
    if(df['Price'][i][-1]=='+'):
        df['Price'][i]=df['Price'][i][0:-1]  
    else:
        df['Price'][i]=df['Price'][i]
        

C:\Users\STSC\AppData\Local\Temp\ipykernel_10052\2055660007.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i]
C:\Users\STSC\AppData\Local\Temp\ipykernel_10052\2055660007.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'][i]=df['Price'][i][0:-1]


In [19]:


df['Price'] = df['Price'].str.replace(',','')
df['Price'] = df['Price'].astype(float)
df.loc[df['Year_Built'] == '', 'Year_Built'] = 0
df['Year_Built'] = pd.to_numeric(df['Year_Built'], errors='coerce')
df['Year_Built'] = df['Year_Built'].fillna(0)
df['Year_Built'] = df['Year_Built'].astype(int)
df.loc[df['Year_Built'] == 0, 'Year_Built'] = np.nan

In [20]:
df.dtypes

State             object
County            object
Street            object
City              object
Zipcode           object
Price            float64
WalkScore          int64
Bike_Score       float64
Beds             float64
Bath             float64
Sqft             float64
Property_Type     object
Year_Built       float64
Status            object
dtype: object

In [21]:
# Normalization

df['Street']=df['Street'].apply(lambda x:x[0]+x[1:].lower())
df['City']=df['City'].apply(lambda x:x[0]+x[1:].lower())
df['State']=df['State'].apply(lambda x:x[0]+x[1:].lower())
df['Property_Type']=df['Property_Type'].apply(lambda x:x[0]+x[1:].lower())
df['Status']=df['Status'].apply(lambda x:x[0]+x[1:].lower())
df

,State,County,Street,City,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,Ca,Marin-County,6 issaquah dock,Sausalito,94965,1495000.0,67,83.0,2.0,2.0,1228.0,Mobile/manufactured home,1987.0,Active
1,Ca,Marin-County,712 tamarack dr s,San rafael,94903,1699000.0,38,27.0,5.0,3.0,2306.0,Single family residential,1963.0,Active
2,Ca,Marin-County,41 bonnie banks way,San rafael,94901,1995000.0,29,29.0,4.0,3.0,3500.0,Single family residential,1970.0,Active
3,Ca,Marin-County,777 grand ave unit -204,San rafael,94901,560000.0,86,79.0,1.0,1.0,1066.0,Condo,1988.0,Active
4,Ca,Marin-County,100 thorndale dr #137,San rafael,94903,275000.0,35,26.0,1.0,1.0,790.0,Condo,1985.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,Ca,Santa-Clara-County,926 northrup st,San jose,95126,1249000.0,82,75.0,2.0,2.0,1934.0,Duplex,1957.0,Active
787,Ca,Santa-Clara-County,4895 pearl ave,San jose,95136,1880000.0,74,74.0,2.0,1.0,1322.0,Single family home,1917.0,Active
788,Ca,Santa-Clara-County,15600 peach hill rd,Saratoga,95070,3749000.0,0,0.0,3.0,2.0,2607.0,Single family home,1960.0,Active
789,Ca,Santa-Clara-County,110 cochrane rd,Morgan hill,95037,920000.0,45,73.0,3.0,3.0,1773.0,Townhouse,2014.0,Active


In [22]:
# Final validation to remove duplicates
df.duplicated().value_counts()
df=df.drop_duplicates()
df

,State,County,Street,City,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,Ca,Marin-County,6 issaquah dock,Sausalito,94965,1495000.0,67,83.0,2.0,2.0,1228.0,Mobile/manufactured home,1987.0,Active
1,Ca,Marin-County,712 tamarack dr s,San rafael,94903,1699000.0,38,27.0,5.0,3.0,2306.0,Single family residential,1963.0,Active
2,Ca,Marin-County,41 bonnie banks way,San rafael,94901,1995000.0,29,29.0,4.0,3.0,3500.0,Single family residential,1970.0,Active
3,Ca,Marin-County,777 grand ave unit -204,San rafael,94901,560000.0,86,79.0,1.0,1.0,1066.0,Condo,1988.0,Active
4,Ca,Marin-County,100 thorndale dr #137,San rafael,94903,275000.0,35,26.0,1.0,1.0,790.0,Condo,1985.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,Ca,Santa-Clara-County,926 northrup st,San jose,95126,1249000.0,82,75.0,2.0,2.0,1934.0,Duplex,1957.0,Active
787,Ca,Santa-Clara-County,4895 pearl ave,San jose,95136,1880000.0,74,74.0,2.0,1.0,1322.0,Single family home,1917.0,Active
788,Ca,Santa-Clara-County,15600 peach hill rd,Saratoga,95070,3749000.0,0,0.0,3.0,2.0,2607.0,Single family home,1960.0,Active
789,Ca,Santa-Clara-County,110 cochrane rd,Morgan hill,95037,920000.0,45,73.0,3.0,3.0,1773.0,Townhouse,2014.0,Active


In [23]:
df.kurt(numeric_only=True)

Price          14.902437
WalkScore      -1.159844
Bike_Score     -1.103569
Beds          193.932392
Bath          203.075625
Sqft          363.720265
Year_Built      0.547584
dtype: float64

In [24]:
df['Price'].describe()

count    7.760000e+02
mean     2.289818e+06
std      3.049998e+06
min      8.500000e+04
25%      7.997492e+05
50%      1.376500e+06
75%      2.378750e+06
max      2.470000e+07
Name: Price, dtype: float64

In [25]:
df['Beds'].describe()

count    776.000000
mean       3.398196
std        1.975318
min        1.000000
25%        3.000000
50%        3.000000
75%        4.000000
max       42.000000
Name: Beds, dtype: float64

In [26]:
df['Bath'].describe()

count    776.000000
mean       2.688789
std        1.930266
min        1.000000
25%        2.000000
50%        2.500000
75%        3.000000
max       41.000000
Name: Bath, dtype: float64

In [27]:
df['Sqft'].describe()

count      776.000000
mean      2372.645619
std       3203.303210
min        532.000000
25%       1321.500000
50%       1777.000000
75%       2533.500000
max      76012.000000
Name: Sqft, dtype: float64

In [28]:
# Calculating Quartiles, IQR and removing the outliers 

In [29]:
Q1_Price = df['Price'].quantile(0.25)
Q3_Price = df['Price'].quantile(0.75)
IQR_Price = Q3_Price - Q1_Price
IQR_Price

1579000.75

In [30]:
Q1_Beds = df['Beds'].quantile(0.25)
Q3_Beds = df['Beds'].quantile(0.75)
IQR_Beds = Q3_Beds - Q1_Beds
IQR_Beds

1.0

In [31]:
Q1_Bath = df['Bath'].quantile(0.25)
Q3_Bath = df['Bath'].quantile(0.75)
IQR_Bath = Q3_Bath - Q1_Bath
IQR_Bath

1.0

In [32]:
Q1_Sqft = df['Sqft'].quantile(0.25)
Q3_Sqft = df['Sqft'].quantile(0.75)
IQR_Sqft = Q3_Sqft - Q1_Sqft
IQR_Sqft

1212.0

In [33]:
df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] > 
                                                                           (Q3_Price + 1.5 * IQR_Price)))]
df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] > 
                                                                           (Q3_Beds + 1.5 * IQR_Beds)))]
df['Bath'] = df['Bath'][~((df['Bath'] < (Q1_Bath - 1.5 * IQR_Bath)) |(df['Bath'] > 
                                                                           (Q3_Bath + 1.5 * IQR_Bath)))]
df['Sqft'] = df['Sqft'][~((df['Sqft'] < (Q1_Sqft - 1.5 * IQR_Sqft)) |(df['Sqft'] > 
                                                                           (Q3_Sqft + 1.5 * IQR_Sqft)))]

C:\Users\STSC\AppData\Local\Temp\ipykernel_10052\2212149449.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Price'] = df['Price'][~((df['Price'] < (Q1_Price - 1.5 * IQR_Price)) |(df['Price'] >
C:\Users\STSC\AppData\Local\Temp\ipykernel_10052\2212149449.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Beds'] = df['Beds'][~((df['Beds'] < (Q1_Beds - 1.5 * IQR_Beds)) |(df['Beds'] >
C:\Users\STSC\AppData\Local\Temp\ipykernel_10052\2212149449.py:5: SettingWithCopyWarning: 
A value is trying to be s

In [34]:
df.isna().sum() # validating the count of outliers removed

State             0
County            0
Street            0
City              0
Zipcode           0
Price            79
WalkScore         0
Bike_Score        0
Beds             84
Bath             53
Sqft             58
Property_Type     0
Year_Built       36
Status            0
dtype: int64

In [35]:
df=df.dropna() #removing the null values. 

In [36]:
df

,State,County,Street,City,Zipcode,Price,WalkScore,Bike_Score,Beds,Bath,Sqft,Property_Type,Year_Built,Status
0,Ca,Marin-County,6 issaquah dock,Sausalito,94965,1495000.0,67,83.0,2.0,2.0,1228.0,Mobile/manufactured home,1987.0,Active
1,Ca,Marin-County,712 tamarack dr s,San rafael,94903,1699000.0,38,27.0,5.0,3.0,2306.0,Single family residential,1963.0,Active
2,Ca,Marin-County,41 bonnie banks way,San rafael,94901,1995000.0,29,29.0,4.0,3.0,3500.0,Single family residential,1970.0,Active
5,Ca,Marin-County,510 midway blvd,Novato,94947,885000.0,21,16.0,3.0,2.0,1212.0,Single family residential,1955.0,Active
6,Ca,Marin-County,848 diablo ave #3,Novato,94947,618000.0,88,69.0,2.0,2.5,1270.0,Condo,2000.0,Active
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786,Ca,Santa-Clara-County,926 northrup st,San jose,95126,1249000.0,82,75.0,2.0,2.0,1934.0,Duplex,1957.0,Active
787,Ca,Santa-Clara-County,4895 pearl ave,San jose,95136,1880000.0,74,74.0,2.0,1.0,1322.0,Single family home,1917.0,Active
788,Ca,Santa-Clara-County,15600 peach hill rd,Saratoga,95070,3749000.0,0,0.0,3.0,2.0,2607.0,Single family home,1960.0,Active
789,Ca,Santa-Clara-County,110 cochrane rd,Morgan hill,95037,920000.0,45,73.0,3.0,3.0,1773.0,Townhouse,2014.0,Active


In [37]:
# Exporting to CSV

In [39]:
df.to_csv('mydata_DA.csv',index=False)